# Import libs

In [ ]:
from __future__ import print_function
import matplotlib.pyplot as plt
%matplotlib inline

import os

import numpy as np
import scipy.io
from models.resnet import ResNet
from models.unet import UNet
from models.skip import skip
import torch
import torch.optim
from PIL import Image
from skimage.metrics import peak_signal_noise_ratio as compare_psnr

import kornia

from utils.inpainting_utils import *


torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark =True
dtype = torch.cuda.FloatTensor

# Choose figure

In [ ]:
file_name  = 'data/inpainting/WDC_Painted.mat'

mat = scipy.io.loadmat(file_name)
img_np = mat["image"]
img_np = img_np.transpose(2,0,1)
img_var = torch.from_numpy(img_np).type(dtype)
mask_np = mat["mask"]
mask_np = mask_np.transpose(2,0,1)
mask_var = torch.from_numpy(mask_np).type(dtype)
painted_np = mat["painted"]
painted_np = painted_np.transpose(2,0,1)
painted_var = torch.from_numpy(painted_np).type(dtype)

# ensure dimensions [0][1] are divisible by 32 (or 2^depth)!

### Visualize

In [ ]:
# band to visualize
band = 20

f, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True, figsize=(15,15))
ax1.imshow(img_var[band,:,:].detach().cpu(), cmap='gray')
ax2.imshow(mask_var[band,:,:].detach().cpu(), cmap='gray') 
ax3.imshow((painted_var)[band,:,:].detach().cpu(), cmap='gray')
plt.show()

# Setup

In [ ]:
pad = 'reflection' #'zero'
OPT_OVER = 'net'
OPTIMIZER = 'adam'

method = '2D'
input_depth = img_np.shape[0] 
LR = 0.01  
num_iter = 10001
param_noise = False
reg_noise_std = 0.1 

show_every = 1000
save_every = 1000

net = skip(input_depth, img_np.shape[0], 
           num_channels_down = [128] * 5, 
           num_channels_up =   [128] * 5,
           num_channels_skip =    [128] * 5,  
           filter_size_up = 3, filter_size_down = 3, 
           upsample_mode='nearest', filter_skip_size=1,
           need_sigmoid=True, need_bias=True, pad=pad, act_fun='LeakyReLU').type(dtype)    

net = net.type(dtype)
net_input = get_noise(input_depth, method, img_np.shape[1:]).type(dtype)

In [ ]:
# Compute number of parameters
s  = sum(np.prod(list(p.size())) for p in net.parameters())
print ('Number of params: %d' % s)

# Loss

delta = 1 # This value is using for measure of robustness. 1 for coverntional inpainting (reducing to DLRHyIn)

HLF = torch.nn.HuberLoss(reduction='sum', delta=delta).type(dtype)

painted_var = painted_var[None, :].cuda()
img_var = img_var[None, :].cuda()
mask_var = mask_var[None, :].cuda()

# Main loop

In [ ]:
f, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True, figsize=(15,15))
ax1.imshow(img_var.detach().cpu().numpy()[0,10,:,:], cmap='gray')
ax2.imshow(img_var.detach().cpu().numpy()[0,170,:,:], cmap='gray')
ax3.imshow(img_var.detach().cpu().numpy()[0,150,:,:], cmap='gray')
plt.show()

def R_eta(input,eta=0.04):

    sing = torch.linalg.svdvals(input)
    rank = torch.sum((1-torch.exp(-sing//eta)))

    return rank

reg_par = 0.1      #1ambda in paper 
        
i = 0
def closure():
    
    global i
    
    if param_noise:
        for n in [x for x in net.parameters() if len(x.size()) == 4]:
            n = n + n.detach().clone().normal_() * n.std() / 50
    
    net_input = net_input_saved
    if reg_noise_std > 0:
        net_input = net_input_saved + (noise.normal_() * reg_noise_std)
         
    out = net(net_input)
    
    total_loss = HLF(out * mask_var, painted_var)+reg_par * R_eta(out.view(input_depth, -1))
    total_loss.backward()
    
    psrn_gt = compare_psnr(img_np, out.detach().cpu().numpy()[0]) 

     
        
    print ('Iteration %05d    Loss %f  PSRN_gt: %f' % (i, total_loss.item(), psrn_gt), '\r', end='')
    if  i % show_every == 0:
        out_np = out.detach().cpu().numpy()[0]
        f, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True, figsize=(15,15))
        ax1.imshow(out_np[10,:,:], cmap='gray')
        ax2.imshow(out_np[150,:,:], cmap='gray')
        ax3.imshow(out_np[180,:,:], cmap='gray')
        plt.show()
        
    if  i % save_every == 0:
        out_np = out.detach().cpu().squeeze().numpy()
        scipy.io.savemat("data/inpainting/results/result_inpainting_2D_it%05d_192.mat" % (i), {'pred':out_np.transpose(1,2,0)})
        
    i += 1

    return total_loss

net_input_saved = net_input.detach().clone()
noise = net_input.detach().clone()

p = get_params(OPT_OVER, net, net_input)
optimize(OPTIMIZER, p, closure, LR, num_iter)